In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.shape

(50000, 2)

In [6]:
sentences = df['review'].tolist()
sentiments = df['sentiment'].tolist()
labels = []

In [7]:
for i in sentiments:
    if i == 'positive':
        i = 1
    else :
        i = 0
    labels.append(i)

In [8]:
training_size = int(len(sentences) * 0.8)

#training_testing data
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

#Make labels into numpy arrays for use with network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [9]:
print(training_sentences[17])

This movie made it into one of my top 10 most awful movies. Horrible. <br /><br />There wasn't a continuous minute where there wasn't a fight with one monster or another. There was no chance for any character development, they were too busy running from one sword fight to another. I had no emotional attachment (except to the big bad machine that wanted to destroy them) <br /><br />Scenes were blatantly stolen from other movies, LOTR, Star Wars and Matrix. <br /><br />Examples<br /><br />>The ghost scene at the end was stolen from the final scene of the old Star Wars with Yoda, Obee One and Vader. <br /><br />>The spider machine in the beginning was exactly like Frodo being attacked by the spider in Return of the Kings. (Elijah Wood is the victim in both films) and wait......it hypnotizes (stings) its victim and wraps them up.....uh hello????<br /><br />>And the whole machine vs. humans theme WAS the Matrix..or Terminator.....<br /><br />There are more examples but why waste the time? A

In [10]:
print(training_labels_final[17])

0


## Tokenize the dataset

In [11]:
vocab_size = 5000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words= vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)


In [12]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

a wonderful little production br br the filming technique is very <OOV> very old time bbc fashion and gives a <OOV> and sometimes <OOV> sense of realism to the entire piece br br the actors are extremely well chosen michael sheen not only has got all the <OOV> but he has all the voices down pat too you can truly see the <OOV> editing <OOV> by the references to <OOV> <OOV> <OOV> not only is it well worth the watching but it is a <OOV> written and performed piece a masterful production about one of the great <OOV> of comedy
A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching b

In [13]:
#Build basic sentiment netwotk
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           80000     
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                51232     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 131,777
Trainable params: 131,777
Non-trainable params: 0
_________________________________________________________________


In [14]:
epochs = 8
model.fit(padded, training_labels_final, epochs=epochs, validation_data = (testing_padded, testing_labels_final))

Epoch 1/8
1250/1250 [==============================] - 5s 3ms/step - loss: 0.5502 - accuracy: 0.6845 - val_loss: 0.3846 - val_accuracy: 0.8270
Epoch 2/8
1250/1250 [==============================] - 3s 3ms/step - loss: 0.2793 - accuracy: 0.8862 - val_loss: 0.4236 - val_accuracy: 0.8110
Epoch 3/8
1250/1250 [==============================] - 3s 3ms/step - loss: 0.1381 - accuracy: 0.9523 - val_loss: 0.5625 - val_accuracy: 0.8004
Epoch 4/8
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0544 - accuracy: 0.9824 - val_loss: 0.8033 - val_accuracy: 0.7861
Epoch 5/8
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0287 - accuracy: 0.9906 - val_loss: 1.0484 - val_accuracy: 0.7832
Epoch 6/8
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0203 - accuracy: 0.9935 - val_loss: 1.2181 - val_accuracy: 0.7864
Epoch 7/8
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0189 - accuracy: 0.9941 - val_loss: 1.3447 - val_accuracy: 0.7915

In [16]:
fake_reviews = ['This movie made it into one of my top 10 most awful movies. Horrible. <br /><br />There wasnt a continuous minute where there wasnt a fight with one monster or another. There was no chance for any character development, they were too busy running from one sword fight to another. I had no emotional attachment (except to the big bad machine that wanted to destroy them) <br /><br />Scenes were blatantly stolen from other movies, LOTR, Star Wars and Matrix. <br /><br />Examples<br /><br />>The ghost scene at the end was stolen from the final scene of the old Star Wars with Yoda, Obee One and Vader. <br /><br />>The spider machine in the beginning was exactly like Frodo being attacked by the spider in Return of the Kings. (Elijah Wood is the victim in both films) and wait......it hypnotizes (stings) its victim and wraps them up.....uh hello????<br /><br />>And the whole machine vs. humans theme WAS the Matrix..or Terminator.....<br /><br />There are more examples but why waste the time? And will someone tell me what was with the Nazis?!?! Nazis???? <br /><br />There was a juvenile story line rushed to a juvenile conclusion. The movie could not decide if it was a childrens movie or an adult movie and wasnt much of either. <br /><br />Just awful. A real disappointment to say the least. Save your money.']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')


['This movie made it into one of my top 10 most awful movies. Horrible. <br /><br />There wasnt a continuous minute where there wasnt a fight with one monster or another. There was no chance for any character development, they were too busy running from one sword fight to another. I had no emotional attachment (except to the big bad machine that wanted to destroy them) <br /><br />Scenes were blatantly stolen from other movies, LOTR, Star Wars and Matrix. <br /><br />Examples<br /><br />>The ghost scene at the end was stolen from the final scene of the old Star Wars with Yoda, Obee One and Vader. <br /><br />>The spider machine in the beginning was exactly like Frodo being attacked by the spider in Return of the Kings. (Elijah Wood is the victim in both films) and wait......it hypnotizes (stings) its victim and wraps them up.....uh hello????<br /><br />>And the whole machine vs. humans theme WAS the Matrix..or Terminator.....<br /><br />There are more examples but why waste the time? A